# 📘 Clase 22: Dependencias

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/heldigard/unaula-IF0100-POO-II/blob/main/notebooks/unidad-03/clase-22-dependencias.ipynb)

## 🎯 Objetivos de Aprendizaje

Al finalizar esta clase, seras capaz de:
- Usar Depends para inyeccion de dependencias
- Crear dependencias reutilizables
- Implementar autenticacion con dependencias
- Usar sub-dependencias
- Manejar dependencias con yield

---

## 📚 Teoria: Sistema de Dependencias

FastAPI tiene un poderoso sistema de **inyeccion de dependencias** que permite:
- Reutilizar codigo compartido
- Implementar autenticacion
- Validar permisos
- Manejar recursos (conexion a BD, etc.)

---

## 💉 Dependencias Basicas

In [ ]:
# ============================================
# DEPENDENCIAS BASICAS
# ============================================

CODIGO = '''
from fastapi import Depends, FastAPI, HTTPException, status
from typing import Optional

app = FastAPI()

# ========== DEPENDENCIA SIMPLE ==========
async def common_parameters(
    q: Optional[str] = None,
    skip: int = 0,
    limit: int = 100
):
    return {"q": q, "skip": skip, "limit": limit}

@app.get("/items/")
async def read_items(commons: dict = Depends(common_parameters)):
    return commons

@app.get("/users/")
async def read_users(commons: dict = Depends(common_parameters)):
    return commons

# ========== DEPENDENCIA CON CLASE ==========
class CommonQueryParams:
    def __init__(
        self,
        q: Optional[str] = None,
        skip: int = 0,
        limit: int = 100
    ):
        self.q = q
        self.skip = skip
        self.limit = limit

@app.get("/products/")
async def read_products(
    commons: CommonQueryParams = Depends(CommonQueryParams)
):
    return commons
'''

print(CODIGO)

---

## 🔐 Autenticacion con Dependencias

In [ ]:
# ============================================
# AUTENTICACION CON DEPENDENCIAS
# ============================================

AUTH_CODE = '''
from fastapi import Depends, FastAPI, HTTPException, status
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from pydantic import BaseModel
from typing import Optional

app = FastAPI()
security = HTTPBearer()

class Usuario(BaseModel):
    id: int
    username: str
    email: str
    rol: str

# Base de datos simulada
usuarios_db = {
    "token123": Usuario(id=1, username="juan", email="juan@test.com", rol="user"),
    "admin456": Usuario(id=2, username="admin", email="admin@test.com", rol="admin"),
}

# ========== DEPENDENCIA DE AUTENTICACION ==========
async def get_current_user(
    credentials: HTTPAuthorizationCredentials = Depends(security)
) -> Usuario:
    token = credentials.credentials
    
    if token not in usuarios_db:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Token invalido",
            headers={"WWW-Authenticate": "Bearer"},
        )
    
    return usuarios_db[token]

# ========== DEPENDENCIA DE PERMISOS ==========
async def require_admin(
    user: Usuario = Depends(get_current_user)
) -> Usuario:
    if user.rol != "admin":
        raise HTTPException(
            status_code=status.HTTP_403_FORBIDDEN,
            detail="Se requieren permisos de administrador"
        )
    return user

# ========== RUTAS PROTEGIDAS ==========

@app.get("/perfil")
async def read_profile(user: Usuario = Depends(get_current_user)):
    """Cualquier usuario autenticado puede acceder."""
    return user

@app.get("/admin/dashboard")
async def admin_dashboard(user: Usuario = Depends(require_admin)):
    """Solo admins pueden acceder."""
    return {"message": f"Bienvenido admin {user.username}"}
'''

print(AUTH_CODE)

---

## 🔄 Dependencias con Yield (Context Manager)

In [ ]:
# ============================================
# DEPENDENCIAS CON YIELD (SETUP/TEARDOWN)
# ============================================

YIELD_CODE = '''
from fastapi import Depends, FastAPI
from typing import Generator

app = FastAPI()

# Simulacion de conexion a BD
class DatabaseConnection:
    def __init__(self):
        self.connected = False
    
    def connect(self):
        self.connected = True
        print("🟢 BD Conectada")
    
    def disconnect(self):
        self.connected = False
        print("🔴 BD Desconectada")
    
    def query(self, sql: str):
        return [{"id": 1, "name": "test"}]

# Dependencia con yield (context manager)
async def get_db() -> Generator[DatabaseConnection, None, None]:
    db = DatabaseConnection()
    
    try:
        db.connect()
        yield db  # Entrega la conexion
    finally:
        db.disconnect()  # Siempre se ejecuta al final

@app.get("/items/")
async def get_items(db: DatabaseConnection = Depends(get_db)):
    # La conexion esta abierta aqui
    result = db.query("SELECT * FROM items")
    return result
    # La conexion se cierra automaticamente
'''

print(YIELD_CODE)

---

**¡Inyecta dependencias, no acoples codigo! 💉**